### Alternate form of the Hamiltonian

In [15]:
from qiskit.opflow import X, Y, Z, I

M_FACTOR = 10

# Create the Problem Hamiltonian

H = (X ^ X ^ I ^ I) + (X ^ I ^ I ^ X) + (I ^ X ^ I ^ X) + (Y ^ I ^ Z ^ I)
H += (Y ^ I ^ I ^ Z) + (I ^ I ^ Z ^ Z) + (Z ^ Y ^ I ^ I) + (Z ^ I ^ X ^ I)
H += (I ^ Y ^ X ^ I) + (I ^ I ^ Y ^ Y) + (I ^ Z ^ Y ^ I) + (I ^ Z ^ I ^ Y)
H += M_FACTOR * ((X ^ I ^ Z ^ I) + (Z ^ X ^ I ^ I) + (Y ^ I ^ I ^ X) + (I ^ Y ^ I ^ Y) + (I ^ Z ^ X ^ I) + (I ^ I ^ Y ^ Z))

In [19]:
# Cell copied from https://qiskit.org/documentation/tutorials/algorithms/05_qaoa.html
from collections import OrderedDict
from qiskit.opflow import StateFn
import numpy as np


def sample_most_likely(state_vector):
    """Compute the most likely binary string from state vector.
    Args:
        state_vector (numpy.ndarray or dict): state vector or counts.
    Returns:
        numpy.ndarray: binary string as numpy.ndarray of ints.
    """
    if isinstance(state_vector, (OrderedDict, dict)):
        # get the binary string with the largest count
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x
    elif isinstance(state_vector, StateFn):
        binary_string = list(state_vector.sample().keys())[0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x
    else:
        n = int(np.log2(state_vector.shape[0]))
        k = np.argmax(np.abs(state_vector))
        x = np.zeros(n)
        for i in range(n):
            x[i] = k % 2
            k >>= 1
        return x

In [16]:
from qiskit import IBMQ

IBMQ.load_account()
provider = IBMQ.get_provider(hub="ibm-q-utokyo", group="internal", project="hirashi-jst")
backend = provider.get_backend("ibmq_qasm_simulator")

ibmqfactory.load_account:WARNING:2023-04-24 17:23:44,526: Credentials are already in use. The existing account in the session will be replaced.


In [17]:
from qiskit_optimization.runtime import QAOAClient
from qiskit.algorithms.optimizers import COBYLA

# Running using Qiskit Runtime
qaoa = QAOAClient(optimizer=COBYLA(), provider=provider, backend=backend)

In [18]:
result = qaoa.compute_minimum_eigenvalue(H)

In [20]:
output = list(sample_most_likely(result.eigenstate))[::-1]
print(f"full output: {output}")

full output: [1, 1, 1, 0]
